In [1]:
#| default_exp classes.DomoPDP

In [5]:
#| exporti
import asyncio
import datetime as dt
import io
import json
from dataclasses import dataclass, field
from enum import Enum, auto

from fastcore.basics import patch_to

import httpx
import pandas as pd

#from ..utils.chunk_execution import chunk_list
#from ..utils.DictDot import DictDot
import domolibrary.utils.DictDot as util_dd
#from . import DomoCertification as dmdc
#from .DomoAuth import DomoDeveloperAuth, DomoFullAuth
import domolibrary.client.DomoAuth as dmda
#from .routes import pdp_routes
import domolibrary.routes.pdp as pdp_routes

In [6]:
#| export
@dataclass
class PDP_Policy():
    dataset_id: str
    filter_group_id: str
    name: str
    resources: list
    parameters: list

    @classmethod
    def _from_json(cls, obj):
        dd = util_dd.DictDot(obj)

        return cls(dataset_id=dd.dataSourceId,
                   filter_group_id=dd.filterGroupId,
                   name=dd.name,
                   resources=dd.resources,
                   parameters=dd.parameters)

    @staticmethod
    def generate_parameter_simple(column_name, column_values_list, operator='EQUALS', ignore_case: bool = True):
        return pdp_routes.generate_policy_parameter_simple(column_name=column_name,
                                                           column_values_list=column_values_list,
                                                           operator=operator,
                                                           ignore_case=ignore_case)

    @staticmethod
    def generate_body(policy_name, dataset_id, parameters_list, policy_id=None, user_ids=None, group_ids=None, virtual_user_ids=None):
        return pdp_routes.generate_policy_body(policy_name=policy_name,
                                               dataset_id=dataset_id,
                                               parameters_list=parameters_list,
                                               policy_id=policy_id,
                                               user_ids=user_ids,
                                               group_ids=group_ids,
                                               virtual_user_ids=virtual_user_ids)

    @classmethod
    async def upsert_policy(cls, 
                            auth: dmda.DomoAuth,
                            dataset_id: str,
                            policy_definition: dict, # body sent to the API (uses camelCase instead of snake_case)
                            debug_api: bool = False,
                            debug_prn: bool = False
                            ):
        
        print(policy_definition)
        
        if policy_definition.get('filterGroupId'):
            if debug_prn:
                print(f'Updating policy in {auth.domo_instance}')    
            res = await pdp_routes.update_policy(auth=auth,
                                                 dataset_id=dataset_id,
                                                 filter_group_id=policy_definition.get(
                                                     'filterGroupId'),
                                                 body=policy_definition,
                                                 debug_api=debug_api)
            return cls._from_json(res.response)
        else:
            if debug_prn:
                print(f'Policy does not exist. Creating policy in {auth.domo_instance}')
            res = await pdp_routes.create_policy(auth=auth,
                                                 dataset_id=dataset_id,
                                                 body=policy_definition,
                                                 debug_api=debug_api)

            return cls._from_json(res.response)
    

#### sample implementation of upsert_policy

In [7]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

pdp_policy = PDP_Policy(
    dataset_id=os.environ["DOJO_DATASET_ID"],
    filter_group_id= '1153',
    name= 'Test Policy Name Employee 123'
)


TypeError: PDP_Policy.__init__() missing 2 required positional arguments: 'resources' and 'parameters'

In [ ]:
#| export
class Dataset_PDP_Policies:

    dataset: dict  # domo dataset class
    policies: list[PDP_Policy] = None  

    def __init__(self, dataset):
        self.dataset = dataset
        self.policies = []

    async def get_policies(self, full_auth: DomoFullAuth = None, dataset_id: str = None, debug: bool = False):

        dataset_id = dataset_id or self.dataset.id
        full_auth = full_auth or self.dataset.full_auth

        res = await pdp_routes.get_pdp_policies(full_auth=full_auth, dataset_id=dataset_id, debug=debug)

        if debug:
            print("debug")
            print(res.status)
            print(res.response)

        if res.status == 200:
            domo_policy = [PDP_Policy._from_json(
                policy_obj) for policy_obj in res.response]
            self.policies = domo_policy
            return domo_policy